In [7]:
# !export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/cuda/lib64
!env CUDA_DEVICE_ORDER=PCI_BUS_ID
!env CUDA_VISIBLE_DEVICES=4

LMOD_DIR=/usr/share/lmod/lmod/libexec/
TMUX=/tmp/tmux-22086/default,1030980,0
CUDA_LAUNCH_BLOCKING=1
SSH_CLIENT=103.107.4.31 63325 22
USER=m13518040
LMOD_COLORIZE=yes
MPLBACKEND=module://matplotlib_inline.backend_inline
LMOD_PKG=/usr/share/lmod/lmod
LD_LIBRARY_PATH=/usr/local/cuda/lib:/usr/local/cuda/lib
HOME=/home/m13518040
MOTD_SHOWN=pam
OLDPWD=/raid/data/m13518040
SSH_TTY=/dev/pts/2
PAGER=cat
TF_CPP_MIN_LOG_LEVEL=1
LMOD_sys=Linux
TF2_BEHAVIOR=1
CUDA_VISIBLE_DEVICES=4,5
LOGNAME=m13518040
TERM=xterm-color
CUDA_DEVICE_ORDER=PCI_BUS_ID
PATH=/home/m13518040/.local/bin:/usr/local/cuda/bin:/opt/bin/:/usr/local/cuda/bin:/home/m13518040/.local/bin:/usr/local/cuda/bin:/opt/bin/:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin
LMOD_FULL_SETTARG_SUPPORT=no
LMOD_PREPEND_BLOCK=normal
KMP_DUPLICATE_LIB_OK=True
KMP_INIT_AT_FORK=FALSE
LANG=en_US.UTF-8
MODULEPATH_ROOT=/sw/modules
SHELL=/bin/sh
LMOD_CMD=/usr/share/lmod/lmod/libexec/

In [8]:
import os, sys
import gc
import swifter

from sklearn.metrics import accuracy_score
import time

gc.collect()
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
# os.environ["CUDA_VISIBLE_DEVICES"] = "4,5"  

import random
import numpy as np
# import jax.numpy as np
import pandas as pd

import torch
from torch import optim
import torch.nn.functional as F
torch.cuda.empty_cache()


import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('omw')

stop_words_set = []
for w in stopwords.words('indonesian'):
    stop_words_set.append(w)

import math
import re
import copy

from operator import itemgetter
from deep_translator import GoogleTranslator

from utils.utils_init_dataset import set_seed, load_dataset_loader
from utils.utils_semantic_use import USE
from utils.utils_data_utils import DocumentSentimentDataset, DocumentSentimentDataLoader, EmotionDetectionDataset, EmotionDetectionDataLoader
from utils.utils_metrics import document_sentiment_metrics_fn
from utils.utils_init_model import text_logit, fine_tuning_model, eval_model, init_model, logit_prob, load_word_index
from utils.utils_forward_fn import forward_sequence_classification

# debugger
from icecream import ic

# !pip install pandarallel
from pandarallel import pandarallel
pandarallel.initialize()

from tqdm.notebook import tqdm
tqdm.pandas()
pd.set_option('display.max_colwidth', None)

# device = torch.device("cuda:2" if torch.cuda.is_available() else "cuda:3")

INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/m13518040/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/m13518040/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw to /home/m13518040/nltk_data...
[nltk_data]   Package omw is already up-to-date!


In [5]:
model_target="IndoBERT"
downstream_task="sentiment"
attack_strategy="adversarial"
# finetune_epoch=1
num_sample=50
# exp_name=
perturbation_technique="codemixing"
perturb_ratio=0.4
perturb_lang="en"
seed=26092020
dataset="valid"

set_seed(seed)
use = USE()

2022-04-16 09:24:20.863288: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib:/usr/local/cuda/lib
2022-04-16 09:24:20.864375: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [9]:
tokenizer, config, finetuned_model = init_model(model_target, downstream_task, seed)
w2i, i2w = load_word_index(downstream_task)

train_dataset, train_loader, train_path = load_dataset_loader(downstream_task, 'train', tokenizer)
valid_dataset, valid_loader, valid_path = load_dataset_loader(downstream_task, 'valid', tokenizer)
test_dataset, test_loader, test_path = load_dataset_loader(downstream_task, 'test', tokenizer)

finetuned_model.to(device)
# finetuned_model = fine_tuning_model(model, i2w, train_loader, valid_loader, finetune_epoch)

if dataset == "valid":
    exp_dataset = valid_dataset.load_dataset(valid_path)[380:430]
elif dataset == "train":
    exp_dataset = train_dataset.load_dataset(train_path)
# exp_dataset = dd.from_pandas(exp_dataset, npartitions=10)
# exp_dataset = te.DataFrame.from_pandas(exp_dataset)
text,label = None,None

RuntimeError: CUDA error: invalid device ordinal

In [ ]:
finetuned_model.eval()
torch.set_grad_enabled(False)

total_loss, total_correct, total_labels = 0, 0, 0
list_hyp, list_label = [], []

pbar = tqdm(test_loader, leave=True, total=len(test_loader))
for i, batch_data in enumerate(pbar):
    ic(batch_data)
    _, batch_hyp, _ = forward_sequence_classification(finetuned_model, batch_data[:-1], i2w=i2w, device='cuda')
    list_hyp += batch_hyp

# Save prediction
df = pd.DataFrame({'label':list_hyp}).reset_index()
df.to_csv('pred.txt', index=False)

print(df)